# 1. Introduction 👋
<center>
<img src="https://images.unsplash.com/photo-1533073526757-2c8ca1df9f1c?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80" alt="Network" width="70%">
</center>
<br>

## Objectives of Notebook 📌
👉 **This notebook aims to:**
*   Describe the definition of decision analysis using decision tree.
*   Demonstrate various examples of decision analysis using R.

---

**Like this notebook? You can support me by giving upvote** 😆👍🔼 <br>
👉 *More about myself: [linktr.ee/caesarmario_](http://linktr.ee/caesarmario_)*

## 1.1 Basic Definitions 📖
> A **decision tree** is a chronological representation of the decision problem. Each decision tree has **two types of nodes**. ***Round nodes*** correspond to the states of nature. While ***square nodes*** correspond to the decision alternatives.<br><br>
The *branches leaving each round node* represent the **different states of nature** while the *branches leaving each square node* represent the **different decision alternatives**.<br><br>
At the *end of each limb* of a tree are the *payoffs* attained from the series of branches making up that limb. <br><br>
The **expected value** of a decision alternative is the sum of weighted payoffs for the decision alternative.

<br>
<center>
<img src="https://i.imgur.com/j3xmXIC.png" alt="DT example" width="50%"><br>
    <i>🖼 Decision Tree Example</i>
</center>
<br>

# 2. Importing Libraries 📚
👉 This section will install some libraries that used to create decision tree diagram.

In [ ]:
install.packages('data.tree')
install.packages('yaml')
install.packages('DiagrammeR')

library(data.tree)
library(yaml)
library(DiagrammeR)

# 3. Decision Analysis
👉 This section will create decision tree diagram, interpret the results from various examples.

## 3.1 Example 1 1️⃣
> Burger Prince Restaurant is contemplating opening a new restaurant on Main Street. It has three different models, each with a different seating capacity. Burger Prince estimates that the average number of customers per hour will be 80, 100, or 120. The payoff table for the three models are described below:

<table>
<thead>
  <tr>
    <th rowspan="2">Model</th>
    <th colspan="3">Average Number of Customers Per Hour</th>
  </tr>
  <tr>
    <th>s1 = 80<br>(p = 0.4)</th>
    <th>s2 = 100<br>(p = 0.2)</th>
    <th>s3 = 120<br>(p = 0.4)</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Model A</td>
    <td>10,000</td>
    <td>15,000</td>
    <td>14,000</td>
  </tr>
  <tr>
    <td>Model B</td>
    <td>8,000</td>
    <td>18,000</td>
    <td>12,000</td>
  </tr>
  <tr>
    <td>Model C</td>
    <td>6,000</td>
    <td>16,000</td>
    <td>21,000</td>
  </tr>
</tbody>
</table>

<br><br>

👉 **Solution:** <br>
Based on the payoff table given, the decision node will be drawn as follows:
<center><img src="https://i.imgur.com/NfrAgOA.png" width="70%"></center><br>

👉 It can be seen that Model C has high EV. <br>
👉 After drawing the decision tree, let's construct the decision tree in `yaml` file.
```
name: Restaurant
type: decision
Model A:
    type: chance
    Customer 80:
        type: terminal
        p: 0.4
        payoff: 10000
    Customer 100:
        type: terminal
        p: 0.2
        payoff: 15000
    Customer 120:
        type: terminal
        p: 0.4
        payoff: 14000
Model B:
    type: chance
    Customer 80:
        type: terminal
        p: 0.4
        payoff: 8000
    Customer 100:
        type: terminal
        p: 0.2
        payoff: 18000
    Customer 120:
        type: terminal
        p: 0.4
        payoff: 12000
Model C:
    type: chance
    Customer 80:
        type: terminal
        p: 0.4
        payoff: 6000
    Customer 100:
        type: terminal
        p: 0.2
        payoff: 16000
    Customer 120:
        type: terminal
        p: 0.4
        payoff: 21000
        
```
👉 Now let's construct the decision tree in R.

In [ ]:
fileName <- '../input/yaml-file/Burger prince.yaml'
l <- yaml.load_file(fileName)
jl <- as.Node(l)
print(jl, "type", "payoff", "p")

In [ ]:
payoff <- function(node) {
  if (node$type == 'chance') node$payoff <- sum(sapply(node$children, function(child) child$payoff * child$p))
  else if (node$type == 'decision') node$payoff <- max(sapply(node$children, function(child) child$payoff))
}

jl$Do(payoff, traversal = "post-order", filterFun = isNotLeaf)

decision <- function(x) {
  po <- sapply(x$children, function(child) child$payoff)
  x$decision <- names(po[po == x$payoff])
}

jl$Do(decision, filterFun = function(x) x$type == 'decision')

GetNodeLabel <- function(node) switch(node$type, 
                                      terminal = paste0( '$ ', format(node$payoff, scientific = FALSE, big.mark = ",")),
                                      paste0('EV/n', '$ ', format(node$payoff, scientific = FALSE, big.mark = ",")))

GetEdgeLabel <- function(node) {
  if (!node$isRoot && node$parent$type == 'chance') {
    label = paste0(node$name, " (", node$p, ")")
  } else {
    label = node$name
  }
  return (label)
}

GetNodeShape <- function(node) switch(node$type, decision = "box", chance = "circle", terminal = "none")


SetEdgeStyle(jl, fontname = 'helvetica', label = GetEdgeLabel)
SetNodeStyle(jl, fontname = 'helvetica', label = GetNodeLabel, shape = GetNodeShape)

jl$Do(function(x) SetEdgeStyle(x, color = "red", inherit = FALSE), 
      filterFun = function(x) !x$isRoot && x$parent$type == "decision" && x$parent$decision == x$name)

SetGraphStyle(jl, rankdir = "LR")

In [ ]:
print(jl, "type", "payoff", "p")

In [ ]:
plot(jl)

👉 From the plot result, it can be seen that model C has highest EV compared to other models.

## 3.2 Example 2 2️⃣
> You are about to introduce fruit gift box, which is a great gift for birthday or to wish someone a fast recovery. You are very confident that this fruit gift box will be a great success and want to introduce it immediately without advertising. <br>
Your business partner, Bambang sees it differently and he suggests launching an advertising campaign. The advertising campaign will cost RM 500 and if successful will produce RM 5,000 revenue. If the campaign is unsuccessful (which there is a 25% chance it would), the revenue is estimated at only RM 1,000. If no advertising is used, the revenue is estimated at RM 3,000 with probability of 0.7 if customers are receptive to this fruit gift box and RM 1,000 with probability of 0.3 if they are not. <br>
If no advertising is used, you are thinking to ask Katie to conduct a market survey. The survey cost is RM 250. The survey will be positive or negative, indicating either a good or poor response from customers. Past experiences suggested that the probability of positive is 0.7. The probabilities when no advertising is used will then be revised based on whether the survey is positive or negative. If the survey is positive, the probability that a customer is receptive to this fruit gift box is 0.85. If the survey is negative, the probability that a customer is receptive to this fruit gift box is 0.45.

<br>

👉 **Solution:** <br>
Based on the scenario above, the decision tree in `yaml` file will be as follows:
```
name: Florist Gift Box
type: decision
Advertisement:
    type: chance
    Success:
        type: terminal
        p: 0.75
        payoff: 4500
    Not success:
        type: terminal
        p: 0.25
        payoff: 500
Market survey:
    type: chance
    Positive:
        type: chance
        p: 0.7
        Receptive:
            type: terminal
            p: 0.85
            payoff: 2750
        Not Receptive:
            type: terminal
            p: 0.15
            payoff: 750
    Negative:
        type: chance
        p: 0.3
        Receptive:
            type: terminal
            p: 0.45
            payoff: 2750
        Not Receptive:
            type: terminal
            p: 0.55
            payoff: 750
```
👉 Now let's construct the decision tree in R.


In [ ]:
fileName <- '../input/yaml-file/Florist Gift Box.yaml'
l <- yaml.load_file(fileName)
jl <- as.Node(l)
print(jl, "type", "payoff", "p")

In [ ]:
payoff <- function(node) {
  if (node$type == 'chance') node$payoff <- sum(sapply(node$children, function(child) child$payoff * child$p))
  else if (node$type == 'decision') node$payoff <- max(sapply(node$children, function(child) child$payoff))
}

jl$Do(payoff, traversal = "post-order", filterFun = isNotLeaf)

decision <- function(x) {
  po <- sapply(x$children, function(child) child$payoff)
  x$decision <- names(po[po == x$payoff])
}

jl$Do(decision, filterFun = function(x) x$type == 'decision')

GetNodeLabel <- function(node) switch(node$type, 
                                      terminal = paste0( 'RM ', format(node$payoff, scientific = FALSE, big.mark = ",")),
                                      paste0('EV/n', 'RM ', format(node$payoff, scientific = FALSE, big.mark = ",")))

GetEdgeLabel <- function(node) {
  if (!node$isRoot && node$parent$type == 'chance') {
    label = paste0(node$name, " (", node$p, ")")
  } else {
    label = node$name
  }
  return (label)
}

GetNodeShape <- function(node) switch(node$type, decision = "box", chance = "circle", terminal = "none")


SetEdgeStyle(jl, fontname = 'helvetica', label = GetEdgeLabel)
SetNodeStyle(jl, fontname = 'helvetica', label = GetNodeLabel, shape = GetNodeShape)

jl$Do(function(x) SetEdgeStyle(x, color = "red", inherit = FALSE), 
      filterFun = function(x) !x$isRoot && x$parent$type == "decision" && x$parent$decision == x$name)

SetGraphStyle(jl, rankdir = "LR")

In [ ]:
print(jl, "type", "payoff", "p")

In [ ]:
plot(jl)

👉 Based on the results of the decision tree compiled, it can be seen that the **EV for the use of advertisements is higher** than the EV of the market survey. <br>
👉 The **EV obtained for advertisement is RM 3,500, while the EV market survey is only RM 2,210**. Furthermore, suppose the gift box business chooses to conduct a market survey. <br>
👉 In that case, **a positive market survey will provide a greater EV than a negative market survey**. <br>
👉 It can be concluded that **the best decision for selling gift boxes based on the decision tree diagram above is the use of advertisements because the EV obtained is higher than the EV market survey**.


# 4. References 🔗
👉 **Book:**
*   [Operations Research: Applications and Algorithms Fourth Edition](https://www.amazon.com/Operations-Research-Applications-Algorithms-InfoTrac/dp/0534380581)


---

**Like this notebook? You can support me by giving upvote** 😆👍🔼 <br>
👉 *More about myself: [linktr.ee/caesarmario_](http://linktr.ee/caesarmario_)* <br><br>

<center>
    <img src="https://i.imgur.com/qLGcpSt.png" alt="WM">
</center>